In [206]:
import pandas as pd
import plotly.express as px
from datetime import datetime, date
dt_string = datetime.now().strftime('%y%m%d')
pd.set_option('display.max_columns', 500)

In [207]:

cd_name = "220721-1648-cf11_cd_21-output"
cd_cf11 = pd.read_excel(f'input/{cd_name}.xlsx', sheet_name='DB', dtype=str)
cd_cf11['origen'] = 'CD'

In [208]:
tienda_name = "220517-0940-cf11_tienda_21-output"
tienda_cf11 = pd.read_excel(f'input/{tienda_name}.xlsx', sheet_name='BD', dtype=str)
tienda_cf11['origen'] = 'TIENDA'

In [209]:
cd_to_del = ['indice_cf11','prd_upc', 'transportadora_nuevo', 'reporte_a_contabilidad', 'movimiento_contable',
        'tranf_electro_factura', 'f3', 'f4', 'f5', 'f11_nuevo', 'f12',
       'nota', 'ro', 'gco_dup', 'dupmc', 'error_ru', 'gco_dupall']
cd_cf11.drop(cd_to_del, axis=1, inplace=True)

In [210]:
tienda_to_del = ['indice_c11t', 'ean', 'propietario', 'f']
tienda_cf11.drop(tienda_to_del, axis=1, inplace=True)

In [211]:
tienda_cf11.rename(columns={'folio_servicio_tecnico':'n_folio', 'estado_servicio_tecnico':'estado_f11',
       'costo_promedio':'costo_total', 'tipo_servicio_f11': 'xservicio', 'motivo_cierre':'status_final', 'cantidad_f11':'qproducto'}, inplace=True)

In [212]:
con = pd.concat([cd_cf11, tienda_cf11])
con['partition_date'] = date.today()
con['costo_total'] = pd.to_numeric(con['costo_total'])
con['status_final'] = con.status_final.str.capitalize()

In [213]:
con['Comentario GCO'].unique()

array(['Coincidencia exacta F4+UPC+QTY', 'Coincidencia exacta F3+UPC+QTY',
       'Registro sin nro. de f11 nuevo', 'Coincidencia exacta (F12|F11)',
       'No coincide (F12|F11)', 'No coincide F5|UPC|Qty',
       'Duplicidad verificada F12+UPC+Cantidad', nan,
       'F11 nuevo no encontrado', 'Coincidencia exacta F5+UPC+QTY',
       'No coincide F3|UPC|QTY', 'F11 nuevo en estado despachado',
       'Registro de f11 nuevo encontrado',
       'Recibido con fecha posterior al 14/01/2022',
       'Duplicidad verificada F11+UPC+Cantidad',
       'Cantidad de los F11s de un F4 > cantidad del F4',
       'Coincidencia F4+UPC+QTY + Registro duplicado en DB',
       'Error de clasificacion - Duplicidad F12+UPC+Cantidad',
       'No coincide F4|UPC|QTY',
       'Coincidencia (F12|F11) + Registro duplicado en DB',
       'Registro duplicado F12 + UPC + Cantidad',
       'Coincidencia F5+UPC+QTY - Se requiere F12|UPC|QTY para validar duplicidad',
       'Cantidad de las F11s de un F5 > cantidad d

In [214]:
con_nov = [
       'Registro sin nro. de f11 nuevo', 
       'No coincide F3|UPC|QTY', 'No coincide (F12|F11)',
       'No coincide F5|UPC|Qty', 'Registro sin nro. de F4',
       'F11 nuevo no encontrado',
       'Recibido con fecha anterior al 14/01/2022', 
       'Recibido con fecha posterior al 14/01/2022', 
       'Registro duplicado F12 + UPC + Cantidad',
       'Coincidencia F4+UPC+QTY + Registro duplicado en DB',
       'Coincidencia (F12|F11) + Registro duplicado en DB',
       'Registro con estado diferente a recibido',
       'Cantidad de los F11s de un F4 > cantidad del F4',
       'No coincide F4|UPC|QTY',
       'Coincidencia F3+UPC+QTY + Registro duplicado en DB',
       'No coincide (F11|F4)+UPC+QTY', 'No coincide (F11|F3)+UPC+QTY',
       'Registro sin nro. de F5',
       'Registro con año de reserva diferente a 2022',
       'La cantidad sumada de los F11s de un F5 es mayor que la cantidad del F5',
       'Registro duplicado F+UPC+Cantidad',
       'Registro con año de confirmación diferente a 2022',
       'Cantidad de los F11s de un F3 > cantidad del F3',
       'Registro anulado','Coincidencia F5+UPC+QTY - Se requiere F12|UPC|QTY para validar duplicidad',
       'Cantidad de las F11s de un F5 > cantidad del F5','Error de clasificacion - Duplicidad F12+UPC+Cantidad']

sin_nov = ['Coincidencia exacta F4+UPC+QTY', 'Coincidencia exacta F3+UPC+QTY',
'Coincidencia exacta (F12|F11)', 'F11 nuevo en estado despachado',
'Coincidencia exacta F5+UPC+QTY', 'Coincidencia (F12|F11) - Se requiere F12|UPC|QTY para validar duplicidad',
'Coincidencia exacta (F11|F3)+UPC+QTY', 'Coincidencia exacta (F11|F4)+UPC+QTY',
'Duplicidad verificada F12+UPC+Cantidad', 'Duplicidad verificada F11+UPC+Cantidad',
 'Error f11','Registro de f11 nuevo encontrado']

con.loc[con['Comentario GCO'].isin(con_nov), 'rev'] = 'Con novedad'
con.loc[con['Comentario GCO'].isin(sin_nov), 'rev'] = 'Sin novedad'
con['rev'] = con['rev'].fillna('Pendiente')


In [215]:
301/4071

0.07393760746745272

In [216]:
con.columns

Index(['nfolio', 'qproducto', 'xobservacion', 'xservicio', 'costo_total',
       'estado_f11', 'status_final', 'sku', 'GCO', 'Comentario GCO', 'checked',
       'fnandup', 'origen', 'n_folio', 'local_envio', 'partition_date', 'rev'],
      dtype='object')

In [217]:
# CD
cd_fi = [ 'Escalamiento li',  'Ingreso con sku trocado', 'Pendiente revisar', 'Ingresa con f12 trocado']
cd_error = ['Error en creacion de f11', 'Con nuevo f11 en estado despachado', 'Error de cierre', 'Error en creacia3n de f11']
con.loc[con.status_final.isin(cd_fi), 'status_final'] = 'Falta infomación CD'
con.loc[con.status_final.isin(cd_error), 'status_final'] = 'Error cierre/creación F11'
duplicados = ['Cierre x duplicidad f11+upc+qty', 'Cierre x duplicidad f12+upc+qty']
con.loc[con.status_final.isin(duplicados), 'status_final'] = 'Duplicados'
con.loc[con.status_final=='Cierre x producto guardado despues de inventario', 'status_final'] = 'Guardado después de inv'
con.loc[con.status_final=='Cierre x producto guardado antes de inventario', 'status_final'] = 'Guardado antes de inv'


con.loc[con.status_final=='Cierre x f3 devuelto a proveedor', 'status_final'] = 'F3 devuelto a proveedor'
con.loc[con.status_final=='Cierre x f4 cobrado a terceros', 'status_final'] = 'F4 cobrado a terceros'


In [218]:
# Tienda
tienda_ct = ['Sin informacion - tienda', 'Duplicado con ajuste en tienda',
       'Entregado en tienda', 'Cambio producto x nuevo', 'Dv','Ajuste inventarios', 
       'Entregado a cliente', 'Validar cierre con chile', 'F11 concesion', 'Nuevo f11']
con.loc[con.status_final.isin(tienda_ct), 'status_final'] = 'Cierre en tienda'
con.loc[con.status_final=='F3', 'status_final'] = 'F3 devuelto a proveedor'
con.loc[con.status_final=='F4 en tienda', 'status_final'] = 'F4 de merma'
con.loc[con.status_final=='F4', 'status_final'] = 'F4 de merma'

In [219]:
con_total = con.groupby('status_final')['costo_total'].sum().sort_values(ascending=False).reset_index()

In [220]:
total_f11s = con_total['costo_total'].sum()

In [221]:
fig_con_total = px.bar(con_total, x ='status_final', y='costo_total', title=f"Costo total de F11s según motivos de cierre - $ {total_f11s/1e6:,.0f}M", labels={ 'costo_total':'Total costo promedio', 'status_final':'Motivo de cierre', 'GRUPO':'Local'}, text='costo_total',text_auto='.2s' ) #category_orders=co ) 
#fig_con_total.update_layout(legend=dict(yanchor="top", y=0.9, xanchor="left", x=0.1))
# fig_con_total.write_image(f'images/{dt_string}_con_total.svg',width=1300, height=300, engine='orca') # 820 , 350 
fig_con_total.show()

In [222]:
con['Comentario GCO'].unique()

array(['Coincidencia exacta F4+UPC+QTY', 'Coincidencia exacta F3+UPC+QTY',
       'Registro sin nro. de f11 nuevo', 'Coincidencia exacta (F12|F11)',
       'No coincide (F12|F11)', 'No coincide F5|UPC|Qty',
       'Duplicidad verificada F12+UPC+Cantidad', nan,
       'F11 nuevo no encontrado', 'Coincidencia exacta F5+UPC+QTY',
       'No coincide F3|UPC|QTY', 'F11 nuevo en estado despachado',
       'Registro de f11 nuevo encontrado',
       'Recibido con fecha posterior al 14/01/2022',
       'Duplicidad verificada F11+UPC+Cantidad',
       'Cantidad de los F11s de un F4 > cantidad del F4',
       'Coincidencia F4+UPC+QTY + Registro duplicado en DB',
       'Error de clasificacion - Duplicidad F12+UPC+Cantidad',
       'No coincide F4|UPC|QTY',
       'Coincidencia (F12|F11) + Registro duplicado en DB',
       'Registro duplicado F12 + UPC + Cantidad',
       'Coincidencia F5+UPC+QTY - Se requiere F12|UPC|QTY para validar duplicidad',
       'Cantidad de las F11s de un F5 > cantidad d

In [223]:
con.loc[con.rev == 'Pendiente']

,nfolio,qproducto,xobservacion,xservicio,costo_total,estado_f11,status_final,sku,GCO,Comentario GCO,checked,fnandup,origen,n_folio,local_envio,partition_date,rev
593,1111259769,1,NaN,retiro f12,12138.000,entrega total,Falta infomación CD,881863466,NaN,NaN,n,NaN,CD,NaN,NaN,2022-07-21,Pendiente
767,1111275073,1,NaN,retiro f12,56700.000,entrega total,Falta infomación CD,8265836,NaN,NaN,n,NaN,CD,NaN,NaN,2022-07-21,Pendiente
2172,1111358868,1,NaN,retiro f12,34148.000,entrega total,Falta infomación CD,881416946,NaN,NaN,n,NaN,CD,NaN,NaN,2022-07-21,Pendiente
2181,1111359001,1,NaN,retiro f12,181827.000,entrega total,Falta infomación CD,10846153,NaN,NaN,n,NaN,CD,NaN,NaN,2022-07-21,Pendiente
2337,1111362346,1,NaN,retiro f12,26595.000,entrega total,Falta infomación CD,13008323,NaN,NaN,n,NaN,CD,NaN,NaN,2022-07-21,Pendiente
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3670,NaN,1,NaN,s.tecnico,1452545.488,entrega total,Sin informacian - tienda,NaN,NaN,NaN,n,NaN,TIENDA,1111647085,25,2022-07-21,Pendiente
3671,NaN,1,NaN,s.tecnico,418598.533,entrega total,Sin informacian - tienda,NaN,NaN,NaN,n,NaN,TIENDA,1111647924,60,2022-07-21,Pendiente
3672,NaN,1,NaN,s.tecnico,1512772.131,entrega total,Sin informacian - tienda,NaN,NaN,NaN,n,NaN,TIENDA,1111647931,50,2022-07-21,Pendiente
3673,NaN,1,NaN,s.tecnico,608913.000,entrega total,Sin informacian - tienda,NaN,NaN,NaN,n,NaN,TIENDA,1111648686,82,2022-07-21,Pendiente


In [224]:
con_cd = con.loc[con.origen =='CD'].groupby(['status_final', 'rev'], sort=False)['costo_total'].sum().sort_values(ascending=False).reset_index()

In [225]:
motivos = con_cd.status_final.unique()
gb_motivos_cd = con_cd.groupby('status_final')['costo_total'].sum().reset_index()
gb_motivos_cd.set_index('status_final', inplace=True)
for motivo in motivos:
    con_cd.loc[con_cd['status_final']==motivo, 'status_final'] = f'{motivo} - ${round(gb_motivos_cd.loc[motivo, "costo_total"]/1e6):,.0f}M'

In [226]:
revs = con_cd.rev.unique()
gb_rev_cd = con_cd.groupby('rev')['costo_total'].sum().reset_index()
gb_rev_cd.set_index('rev', inplace=True)
for rev in revs:
    con_cd.loc[con_cd['rev']==rev, 'rev'] = f'{rev} - ${round(gb_rev_cd.loc[rev, "costo_total"]/1e6):,.0f}M'

In [227]:
revs =  [con_cd.loc[con_cd['rev'].str.contains(r'Sin novedad'), 'rev'].unique()[0], con_cd.loc[con_cd['rev'].str.contains(r'Con novedad'), 'rev'].unique()[0], con_cd.loc[con_cd['rev'].str.contains(r'Pendiente'), 'rev'].unique()[0]]
co_cd = {'rev': revs, 'status_final':con_cd.status_final.unique()}

In [228]:
total_cd= con_cd['costo_total'].sum()

In [229]:
con_cd

,status_final,rev,costo_total
0,Cierre por producto guardado despues de invent...,"Sin novedad - $5,557M",2.176082e+09
1,"Cierre por f4 cobrado a terceros - $1,996M","Sin novedad - $5,557M",1.986231e+09
2,Cierre por f3 devuelto a proveedor - $868M,"Sin novedad - $5,557M",8.674882e+08
3,F4 de merma - $324M,"Sin novedad - $5,557M",3.233581e+08
4,Error cierre/creación F11 - $170M,Con novedad - $363M,1.589833e+08
5,Cierre por producto guardado despues de invent...,Con novedad - $363M,1.315444e+08
6,Duplicados - $130M,"Sin novedad - $5,557M",1.261273e+08
7,Producto compensado con tienda - recibido - $64M,"Sin novedad - $5,557M",6.108006e+07
8,Cierre por producto guardado antes de inventar...,Con novedad - $363M,5.503432e+07
9,Falta infomación CD - $46M,Pendiente - $47M,4.565560e+07


In [239]:
fig_con_total = px.bar(con_cd, orientation='h', y ='status_final', x='costo_total', color='rev', category_orders=co_cd, title= f"Motivos de cierres de F11 CD 2022 - $ {total_cd/1e6:,.0f}M", 
labels={ 'costo_total':'Total costo promedio', 'status_final':'Motivo de cierre', 'rev':'Estado revisión'}, text='costo_total',text_auto='.2s', 
color_discrete_sequence=[ 'rgb(115, 175, 72)','rgb(204, 80, 62)','rgb(179, 179, 179)' ] ) 
fig_con_total.update_layout(legend=dict(yanchor="bottom", y=0.1, xanchor="right", x=0.9))
fig_con_total.update_layout(margin_r=20, margin_t=60)
fig_con_total.write_image(f'output/{dt_string}_con_cd.svg',width=800, height=400, engine='orca') # 820 , 350 
fig_con_total.show()

In [231]:
con_tienda = con.loc[con.origen =='TIENDA'].groupby(['status_final', 'rev'])['costo_total'].sum().sort_values(ascending=False).reset_index()

In [232]:
motivos = con_tienda.status_final.unique()
gb_motivos_tienda = con_tienda.groupby('status_final')['costo_total'].sum().reset_index()
gb_motivos_tienda.set_index('status_final', inplace=True)
for motivo in motivos:
    con_tienda.loc[con_tienda['status_final']==motivo, 'status_final'] = f'{motivo} - ${round(gb_motivos_tienda.loc[motivo, "costo_total"]/1e6):,.0f}M'

In [233]:
revs = con_tienda.rev.unique()
gb_rev_tienda = con_tienda.groupby('rev')['costo_total'].sum().reset_index()
gb_rev_tienda.set_index('rev', inplace=True)
for rev in revs:
    con_tienda.loc[con_tienda['rev']==rev, 'rev'] = f'{rev} - ${round(gb_rev_tienda.loc[rev, "costo_total"]/1e6):,.0f}M'

In [234]:
revs_tienda =  [con_tienda.loc[con_tienda['rev'].str.contains(r'Sin novedad'), 'rev'].unique()[0], con_tienda.loc[con_tienda['rev'].str.contains(r'Con novedad'), 'rev'].unique()[0], con_tienda.loc[con_tienda['rev'].str.contains(r'Pendiente'), 'rev'].unique()[0]]
co_tienda = {'rev':revs_tienda, 'status_final':con_tienda.status_final.unique()}

In [235]:
total_tienda = con_tienda['costo_total'].sum()

In [236]:
fig_con_total = px.bar(con_tienda, orientation='h', y ='status_final', x='costo_total', title= f"Motivos de cierres de F11 Tienda 2022 - $ {total_tienda/1e6:,.0f}M", 
labels={ 'costo_total':'Total costo promedio', 'status_final':'Motivo de cierre', 'rev':'Estado revisión'}, text='costo_total',text_auto='.2s', 
color_discrete_sequence=[ 'rgb(115, 175, 72)','rgb(204, 80, 62)','rgb(179, 179, 179)' ], color ='rev', category_orders=co_tienda ) 
fig_con_total.update_layout(legend=dict(yanchor="bottom", y=0.1, xanchor="right", x=0.9))
fig_con_total.update_layout(margin_r=20, margin_t=60)
# fig_con_total.write_image(f'images/{dt_string}_con_tienda.svg',width=700, height=400, engine='orca') # 820 , 350 
fig_con_total.show()

In [237]:
con.head()

,nfolio,qproducto,xobservacion,xservicio,costo_total,estado_f11,status_final,sku,GCO,Comentario GCO,checked,fnandup,origen,n_folio,local_envio,partition_date,rev
0,1110454448,2,se genera f4 asociado a 1110454448 y 126301039...,retiro f12,29334.0,entrega total,F4 de merma,9181931,OKK,Coincidencia exacta F4+UPC+QTY,y,NaN,CD,NaN,NaN,2022-07-21,Sin novedad
1,1110454448,2,se genera f4 asociado a 1110454448 y 126301039...,retiro f12,29334.0,entrega total,F4 de merma,9181933,OKK,Coincidencia exacta F4+UPC+QTY,y,NaN,CD,NaN,NaN,2022-07-21,Sin novedad
2,1110484339,1,se genera f4 asociado a 1110484339 y 126070426...,retiro f12,1210950.0,entrega total,F4 de merma,3225964,OKK,Coincidencia exacta F4+UPC+QTY,y,NaN,CD,NaN,NaN,2022-07-21,Sin novedad
3,1110544224,1,se genera f4 asociado a 1110544224 y 12620043...,retiro f12,165900.0,entrega total,F4 de merma,9988778,OKK,Coincidencia exacta F4+UPC+QTY,y,NaN,CD,NaN,NaN,2022-07-21,Sin novedad
4,1110580802,1,se genera f4 asociado a 1110580802 y 12619745...,retiro f12,2544730.0,entrega total,F4 de merma,3394545,OKK,Coincidencia exacta F4+UPC+QTY,y,NaN,CD,NaN,NaN,2022-07-21,Sin novedad


In [238]:
con.to_excel('consolidado_cf11.xlsx', index=False)